### one-hotでは未知の初期値に対応できないので、スカラで表して解決する？
* 元々スカラで状態を表すと、xの値が大きいほど評価値が高く勾配に大きく影響を与えるという問題が起こる。  
xの値が大きいほど、[-1, /2]が大きく評価されるという関係はある。これをどう表現する？

* 状態ベクトルを$s = [x_{now}, x_{init}]^T $としても線形モデルで存在した、$x_{now}$に関わらず$x_{init}$によって評価値が高くなる

### 学習率の調整はいらないのか？

In [24]:
import numpy as np
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

from scipy.special import softmax
from matplotlib import cm

In [25]:
def x2vec(x):
    vec = np.zeros(11)
    vec[x] = 1
    return vec

def vec2x(vec):
    return np.argmax(vec)

def state_vec(s, s0):
    return np.r_[x2vec(s), 1]

In [26]:
def next_state(t, a):
    s, s0 = t
    if a == 0:
        s_next = s * 2
    elif a == 1:
        s_next = s + 1
    elif a == 2:
        s_next = s
    elif a == 3:
        s_next = s - 1
    elif a == 4:
        s_next = s // 2
    s_next = max(0, min(10, s_next))
    return (s_next, s0)

In [27]:
def reward_func(t, t_next, done, T):  # 加藤の実験コードを反映
    s, s0 = t
    s_next, _ = t_next
    if done and s_next != T:
        return -100
    elif done:
        return 100
    return -1

In [28]:
def step(t, a, T):
    t_next = next_state(t, a)
    done = (a == 2)
    r = reward_func(t, t_next, done, T)
    return t_next, r, done 

In [34]:
def update(Q, batch, lr=0.1, gamma=0.9):
    #grad = np.zeros_like(Q)
    states = np.stack([e.s for e in batch])
    estimations = np.zeros((len(batch), Q.shape[1]))
    truth = np.zeros((len(batch), Q.shape[1]))
    for i, e in enumerate(batch):
        u = state_vec(e.s[0], e.s[1])
        est = u @ Q
        true = est.copy()
        v = e.r
        if not e.d:
            u_next = state_vec(e.ns[0], e.ns[1])
            v += gamma * np.max(u_next @ Q)
        true[e.a] = v
        estimations[i] = est
        truth[i] = true
    grad =  np.sum(states.T @ (estimations - truth))
    grad = grad / len(batch)
    Q -= lr * grad

In [30]:
from collections import defaultdict, namedtuple, deque
import random

def f(x):
    #return 0
    return 0

Experience = namedtuple('Experience', ('s', 'a', 'ns', 'r', 'd'))
#starts=[0, 3, 5, 8, 10]
starts=list(range(11))

def trial(seed=100, try_num=100000, lr=0.01, eps=0.1):
    # training
    np.random.seed(seed)
    random.seed(seed)
    #Q = np.zeros((13, 5))
    Q = np.zeros((12, 5))
    R = []
    # each element is [s, a, ns, r, d]
    experiences = deque(maxlen=1024)
    
    for itr in tqdm(range(try_num)):
        Ri = 0
        s0 = np.random.choice(starts)
        T = f(s0)
        t = (s0, s0)
        for _ in range(1000):
            # 行動を選択
            u = state_vec(t[0], t[1])
            a = np.argmax(Q.T @ u) if np.random.rand() >= eps else np.random.randint(5)
            t_next, r, d = step(t, a, T)
            e = Experience(t, a, t_next, r, d)
            experiences.append(e)
            t = t_next
            Ri = Ri + r
            if d:
                break
        if len(experiences) == 1024:
            batch = random.sample(experiences, 32)
            update(Q, batch, lr=lr, gamma=0.9)

        R.append(Ri)
    return Q, R

In [31]:
def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

act = ['*2', '+1', 'end', '-1', '/2']

def viewQ(Q):
    for init in range(11):
        U = np.zeros((11, 5))
        for i in range(11):
            u = state_vec(i, init)
            U[i, :] = softmax(u @ Q)
        plt.imshow(U)
        plt.show()

def view(Q, R, n=1000):
    plt.subplot(121)
    plt.plot(moving_average(R, n=n))
    plt.xlabel('# of Trials')
    plt.ylabel('Avg. Reward')

    table = []
    for s0 in range(11):
        s0_vec = x2vec(s0)
        s = state_vec(s0_vec, s0/10)
        u = s.T @ Q
        table.append(softmax(u))
        print('s = %2d, T= %2d, opt_a = %s ' % (s0, f(s0), act[np.argmax(u)]))
        print(u)

    table = np.array(table)
    ax = plt.subplot(122)
    im = plt.imshow(table, cmap=cm.RdYlGn)
    plt.colorbar(im)
    ax.set_xticks(range(len(act)))
    ax.set_xticklabels(act)

    plt.show()

In [32]:
def test(Q):
    solved = []
    for s0 in range(11):
        T = f(s0)
        s0_vec = x2vec(s0)
        s_vec = state_vec(s0_vec, s0/10)
        for _ in range(100):
            a = np.argmax(s_vec.T @ Q)
            if a == 2:
                break
            s_vec = next_state(s_vec, a)
        if vec2x(s_vec[:11]) == T:
            solved.append((s0, T, 'solved!'))
        else:
            solved.append((s0, T, '-'))
    for item in solved:
        print('init = %d, T = %d, %s' % item)

In [35]:
Q, R = trial(seed=0, eps=0.1, try_num=10000, lr=0.01)

In [19]:
#viewQ(Q)

In [36]:
# 0, 10の奇跡を追ってみる
done = False
max_iter = 10
for init in range(11):
    print('T =', f(init))
    print('*' * 10)
    t = (init, init)
    for i in range(max_iter):
        u = state_vec(t[0], t[1])
        a = np.argmax(u @ Q)
        print('t', t[0], 'a', act[a])
        if a == 2:
            break
        t = next_state(t, a)
    print()

T = 0
**********
t 0 a *2
t 0 a *2
t 0 a *2
t 0 a *2
t 0 a *2
t 0 a *2
t 0 a *2
t 0 a *2
t 0 a *2
t 0 a *2

T = 0
**********
t 1 a *2
t 2 a *2
t 4 a *2
t 8 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2

T = 0
**********
t 2 a *2
t 4 a *2
t 8 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2

T = 0
**********
t 3 a *2
t 6 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2

T = 0
**********
t 4 a *2
t 8 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2

T = 0
**********
t 5 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2

T = 0
**********
t 6 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2

T = 0
**********
t 7 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2

T = 0
**********
t 8 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10 a *2
t 10